In [59]:
# editting on feb 12

In [35]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

In [36]:
adata = read_h5ad('/Users/madelinepark/Downloads/Limb_Muscle_facs.h5ad')
tiss = adata

In [60]:
def loc_split(tiss, feature):
    tiss.obs['feature_type_of_interest'] = 'rest'
    results_feature_cv = pd.DataFrame()
    for c in list(set(tiss.obs[feature])): 
#         print(c)
#         clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
#         selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration  
        feature_of_interest = c
        tiss.obs.loc[tiss.obs[tiss.obs[feature] == feature_of_interest].index,'feature_type_of_interest'] = feature_of_interest
        feat_labels = tiss.var_names 
        X = tiss.X
        y = tiss.obs['feature_type_of_interest']
#         the probem is that we define X here, but use it later 
    return X, y, feature, feat_labels

In [61]:
def train(X, y, test_size, random_state):
# def train(test_size, random_state):
    print('training...')
    feat_labels = loc_split(tiss=tiss, feature='age')[3] 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0) 
    clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration
    clf.fit(X_train, y_train)
    selector.fit(X_train, y_train)
    print(type(feat_labels)) #this is currently a tuple, but it should be a base index i
    feature_selected = feat_labels[selector.support_]
    return feat_labels, feature_selected, selector.estimator_.feature_importances_, X_train, X_test, y_train, y_test

In [62]:
# def result_write (c, feature_selected, tiss, feature_type_of_interest): #feature_selected comes from above function... hm
#     print('result writing')
#     feat_labels = train(X, y, test_size, random_state)[0] #or put as loc_split from above 
#     selector.estimator_.feature_importances_ = train(X, y, test_size, random_state)[2]
#     column_headings = []
#     column_headings.append(c)
#     column_headings.append(c + '_gini')
#     resaux = pd.DataFrame(columns=column_headings)
#     resaux[c] = feature_selected
#     resaux[c + '_gini'] = (selector.estimator_.feature_importances_)
#     print(feature_selected)
#     print (selector.estimator_.feature_importances_)
#     results_feature_cv = pd.concat([results_feature_cv,resaux],axis=1)
#     tiss.obs['feature_type_of_interest'] = 'rest' #this line is to reset the loop, but is not specific to this specific sub function, just didn't know where to put

In [63]:
def result_write (c, feature_selected, tiss): #feature_selected comes from above function... hm
    print('result writing')
    feat_labels = train(X, y, test_size, random_state)[0] #or put as loc_split from above 
    selector.estimator_.feature_importances_ = train(X, y, test_size, random_state)[2]
    column_headings = []
    column_headings.append(c)
    column_headings.append(c + '_gini')
    resaux = pd.DataFrame(columns=column_headings)
    resaux[c] = feature_selected
    resaux[c + '_gini'] = (selector.estimator_.feature_importances_)
    print(feature_selected)
    print (selector.estimator_.feature_importances_)
    results_feature_cv = pd.concat([results_feature_cv,resaux],axis=1)
    tiss.obs['feature_type_of_interest'] = 'rest' #this line is to reset the loop, but is not specific to this specific sub function, just didn't know where to put

In [66]:
def scRFE (tiss=tiss, X=tiss.X,feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5) :
#     clf, selector = make_forest(n_estimators=10, random_state=0, oob_score=True, step=0.2, cv=2, n_jobs=-1)
    for c in list(set(tiss.obs[feature])): 
        print(c)
        X = loc_split(tiss=tiss, feature= 'age')[0]
        y = loc_split(tiss=tiss, feature= 'age')[1]
        feature = loc_split(tiss=tiss, feature= 'age')[2]
        feat_labels = loc_split(tiss=tiss, feature= 'age')[3]
        feature_selected = train(X, y, test_size, random_state)[1]
#         selector.estimator_.feature_importances_ = make_forest (n_estimators, random_state, oob_score, step, cv, n_jobs)[1]
        X_train = train(X, y, test_size, random_state)[2]
        X_test = train(X, y, test_size, random_state)[3]
        y_train = train(X, y, test_size, random_state)[4]
        y_test = train(X, y, test_size, random_state)[5]
        result_write (c, feature_selected, tiss)

In [67]:
scRFE(tiss=tiss, feature='age', n_estimators=10, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5)

24m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

<class 'pandas.core.indexes.base.Index'>
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

<class 'pandas.core.indexes.base.Index'>
training...


...

<class 'pandas.core.indexes.base.Index'>
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

<class 'pandas.core.indexes.base.Index'>
result writing


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


NameError: name 'X' is not defined